# Lesson 11 - SQL Data Cleaning

Useful list of functions: https://www.postgresql.org/docs/8.1/static/functions-string.html

## `LEFT`, `RIGHT`, `LENGTH`

<img src="../SQL/ERD DAND.jpg" width="600" height="400">

This operates exactly as LEFT, RIGHT and LEN in Excel. You can extract characters from a string using those functions. 

**Examples:**

In the accounts table, there is a column holding the website for each company. The last three digits specify what type of web address they are using. A list of extensions (and pricing) is provided here. Pull these extensions and provide how many of each website type exist in the accounts table.

`SELECT RIGHT(website, 3) AS domain, COUNT(*)
FROM accounts
GROUP BY domain
ORDER BY 2 DESC`


There is much debate about how much the name (or even the first letter of a company name) matters. Use the accounts table to pull the first letter of each company name to see the distribution of company names that begin with each letter (or number). 

`SELECT LEFT(UPPER(name), 1) AS first_letter, COUNT(*)
FROM accounts
GROUP BY first_letter
ORDER BY 2 DESC`


Use the accounts table and a CASE statement to create two groups: one group of company names that start with a number and a second group of those company names that start with a letter. What proportion of company names start with a letter?

`There are 350 company names that start with a letter and 1 that starts with a number. This gives a ratio of 350/351 that are company names that start with a letter or 99.7%.
SELECT SUM(num) nums, SUM(letter) letters
FROM (SELECT name, CASE WHEN LEFT(UPPER(name), 1) IN ('0','1','2','3','4','5','6','7','8','9') 
                       THEN 1 ELSE 0 END AS num, 
         CASE WHEN LEFT(UPPER(name), 1) IN ('0','1','2','3','4','5','6','7','8','9') 
                       THEN 0 ELSE 1 END AS letter
      FROM accounts) t1;`
      
      
Consider vowels as a, e, i, o, and u. What proportion of company names start with a vowel, and what percent start with anything else?

`There are 80 company names that start with a vowel and 271 that start with other characters. Therefore 80/351 are vowels or 22.8%. Therefore, 77.2% of company names do not start with vowels.
SELECT SUM(vowels) vowels, SUM(other) other
FROM (SELECT name, CASE WHEN LEFT(UPPER(name), 1) IN ('A','E','I','O','U') 
                        THEN 1 ELSE 0 END AS vowels, 
          CASE WHEN LEFT(UPPER(name), 1) IN ('A','E','I','O','U') 
                       THEN 0 ELSE 1 END AS other
         FROM accounts) t1;`



## `POSITION`

This takes a character and a column, and provides the index where that character is for each row. The index of the first position is 1 in SQL. This is like using the SEARCH function in Excel and determining where it occurs in a string. Here's an example of looking for where a comma starts in the string city_state: `POSITION(',' IN city_state)`. **This is case sensitive.**

## `STRPOS`

`STRPOS` provides the same result as `POSITION` but the syntax for achieving those results is a bit different as shown here: `STRPOS(city_state, ',')`. **This is case sensitive.**

## `UPPER` and `LOWER`

If you want to pull an index regardless of case, you can change the letters to all uppercase or lower case with `UPPER` or `LOWER`.



**Examples:**
    
Use the accounts table to create first and last name columns that hold the first and last names for the primary_poc. 

`SELECT LEFT(primary_poc, POSITION(' ' IN primary_poc)-1) AS first_name, RIGHT(primary_poc,(LENGTH(primary_poc)- STRPOS(primary_poc, ' '))) AS last_name, primary_poc
FROM accounts`

Now see if you can do the same thing for every rep name in the sales_reps table. Again provide first and last name columns.

`SELECT LEFT(name, POSITION(' ' IN name)-1) AS first_name, RIGHT(name,(LENGTH(name)- STRPOS(name, ' '))) AS last_name
FROM sales_reps`


## `CONCAT` and `||`

This involves concatenating columns. You can use `CONCAT(first_name. ' ', last_name)` or with piping as `first_name || '' || last_name`.

**Examples:**

Each company in the accounts table wants to create an email address for each primary_poc. The email address should be the first name of the primary_poc . last name primary_poc @ company name .com.

`SELECT CONCAT(first_name, '.', last_name, '@', name, '.com') AS email_address 
FROM (
    SELECT name, LEFT(primary_poc, POSITION(' ' IN primary_poc)-1) AS first_name, RIGHT(primary_poc,(LENGTH(primary_poc)- STRPOS(primary_poc, ' '))) AS last_name, primary_poc
FROM accounts) sub`


You may have noticed that in the previous solution some of the company names include spaces, which will certainly not work in an email address. See if you can create an email address that will work by removing all of the spaces in the account name, but otherwise your solution should be just as in question 1. Some helpful documentation is here.


`SELECT CONCAT(first_name, '.', last_name, '@', REPLACE(name, ' ', ''), '.com') AS email_address 
FROM (
    SELECT name, LEFT(primary_poc, POSITION(' ' IN primary_poc)-1) AS first_name, RIGHT(primary_poc,(LENGTH(primary_poc)- STRPOS(primary_poc, ' '))) AS last_name, primary_poc
FROM accounts) sub`


We would also like to create an initial password, which they will change after their first log in. The first password will be the first letter of the primary_poc's first name (lowercase), then the last letter of their first name (lowercase), the first letter of their last name (lowercase), the last letter of their last name (lowercase), the number of letters in their first name, the number of letters in their last name, and then the name of the company they are working with, all capitalized with no spaces.

`WITH t1 AS (
 SELECT LEFT(primary_poc,     STRPOS(primary_poc, ' ') -1 ) first_name,  RIGHT(primary_poc, LENGTH(primary_poc) - STRPOS(primary_poc, ' ')) last_name, name
 FROM accounts)
SELECT first_name, last_name, CONCAT(first_name, '.', last_name, '@', name, '.com'), LEFT(LOWER(first_name), 1) || RIGHT(LOWER(first_name), 1) || LEFT(LOWER(last_name), 1) || RIGHT(LOWER(last_name), 1) || LENGTH(first_name) || LENGTH(last_name) || REPLACE(UPPER(name), ' ', '')
FROM t1; `

## `CAST`

You use CAST to convert data types. Commonly from a string to date. For example, `CAST(date_column AS DATE)`. Alternatively, you can also use `date_column::DATE`. Note that using `LEFT`, `RIGHT`, `TRIM` converts extracted data automatically as a string.

## `SUBSTR`

This acts like the MID function in Excel where you extract a string from certain positions - `SUBSTR(string, from position count, how many characters)`.

**Example:**  You want to convert a date in a column that has 01/31/2014 08:00:00AM +000

`SELECT date orig_date, (SUBSTR(date, 7, 4) || '-' || LEFT(date, 2) || '-' || SUBSTR(date, 4, 2))::DATE new_date
FROM sf_crime_data;`

## `COALESCE`

This function returns the first non-null value passed for each row. This is frequently used to replace null values with zero or another value. For example, the below replaces blank columns as zero and fills in missing id numbers which resulted from a `LEFT JOIN`:

`SELECT COALESCE(a.id, a.id) filled_id, a.name, a.website, a.lat, a.long, a.primary_poc, a.sales_rep_id, COALESCE(o.account_id, a.id) account_id, o.occurred_at, COALESCE(o.standard_qty, 0) standard_qty, COALESCE(o.gloss_qty,0) gloss_qty, COALESCE(o.poster_qty,0) poster_qty, COALESCE(o.total,0) total, COALESCE(o.standard_amt_usd,0) standard_amt_usd, COALESCE(o.gloss_amt_usd,0) gloss_amt_usd, COALESCE(o.poster_amt_usd,0) poster_amt_usd, COALESCE(o.total_amt_usd,0) total_amt_usd
FROM accounts a
LEFT JOIN orders o
ON a.id = o.account_id;`

Also look here: https://www.w3schools.com/sql/sql_isnull.asp. It covers `IFNULL()`, `ISNULL()`, `NVL()`

And here from Mode Analytics: https://community.modeanalytics.com/sql/tutorial/sql-string-functions-for-cleaning/
